# Feature Engineering

>The following codes are demos only. It's **NOT for production** due to system security concerns, please **DO NOT** use it directly in production.

It is recommended to use [jupyter](https://jupyter.org/) to run this tutorial.

Secretflow provides a variety of tools to analyze the statistics of the dataset in order to improve the quality of results from the machine learning process.

## Preparation

Initialize secretflow and create two parties alice and bob.

> 💡 Before using preprocessing, you may need to get to know secretflow's [DataFrame](../preprocessing/DataFrame.ipynb).

In [13]:
import secretflow as sf

# In case you have a running secretflow runtime already.
sf.shutdown()

sf.init(['alice', 'bob'], address='local')
alice = sf.PYU('alice')
bob = sf.PYU('bob')

spu = sf.SPU(sf.utils.testing.cluster_def(['alice', 'bob']))

## Data Preparation

Here we use linear as example data.

In [14]:
from secretflow.utils.simulation.datasets import load_linear

vdf = load_linear(parts={alice: (1, 4), bob: (18, 22)})

label_data = vdf["y"]
vdf = vdf.drop(columns="y")

print(f"orig ds in alice:\n {sf.reveal(vdf.partitions[alice].data)}")
print(f"orig ds in bob:\n {sf.reveal(vdf.partitions[bob].data)}")

orig ds in alice:
             x1        x2        x3
0    -0.514226  0.730010 -0.730391
1    -0.725537  0.482244 -0.823223
2     0.608353 -0.071102 -0.775098
3    -0.686642  0.160470  0.914477
4    -0.198111  0.212909  0.950474
...        ...       ...       ...
9995 -0.367246 -0.296454  0.558596
9996  0.010913  0.629268 -0.384093
9997 -0.238097  0.904069 -0.344859
9998  0.453686 -0.375173  0.899238
9999 -0.776015 -0.772112  0.012110

[10000 rows x 3 columns]
orig ds in bob:
            x18       x19       x20
0     0.810261  0.048303  0.937679
1     0.312728  0.526637  0.589773
2     0.039087 -0.753417  0.516735
3    -0.855979  0.250944  0.979465
4    -0.238805  0.243109 -0.121446
...        ...       ...       ...
9995 -0.847253  0.069960  0.786748
9996 -0.502486 -0.076290 -0.604832
9997 -0.424209  0.434947  0.998955
9998  0.914291 -0.473056  0.616257
9999 -0.602927 -0.021368  0.885519

[10000 rows x 3 columns]


## Pearson product-moment correlation coefficient

The Pearson product-moment correlation coefficient is used to measure the degree of correlation (linear correlation) between two variables X and Y.

The Pearson product-moment correlation coefficient between two variables is defined as the covariance of the two variables divided by the product of their standard deviations:

$$ \rho_{X,Y}=\frac{cov(X, Y)}{\sigma_X \sigma_Y}=\frac{(X-\mu_X)(Y-\mu_Y)}{\sigma_X \sigma_Y} $$

$$ \mu_X= \mathbb{E}(X), \sigma_X^2=\mathbb{E}[(X-\mathbb{E}(X))^2]=\mathbb{E}(X^2)-\mathbb{E}^2(X) $$

The Pearson product-moment correlation coefficient for samples(features) from dataset, usually represented by the lowercase letter r:

$$ r=\frac{\sum^n_{i=1}(X_i-\bar{X})(Y_i-\bar{Y})}{\sqrt{\sum^n_{i=1}(X_i-\bar{X})^2} \sqrt{\sum^n_{i=1}(Y_i-\bar{Y})^2}} $$

Its value is between -1 and 1. $r>0$ corresponds to a positive correlation between features, otherwise it is a negative correlation; the larger the $|r|$, the greater the degree of correlation.

### SSVertPearsonR

SecretFlow provides `SSVertPearsonR` for calculating Pearson product-moment correlation coefficient of Vertical DataFrame using secret sharing.

SSVertPearsonR will standardize input dataset first. After standardized, all features' variance is 1 and the mean is 0, the calculation is simplified to:

$$ r=\frac{1}{n-1}X^TX $$



In [15]:
from secretflow.stats import SSVertPearsonR

v_pearsonr = SSVertPearsonR(spu)
corr = v_pearsonr.pearsonr(vdf)
print(f"corr: \n {corr}")

corr: 
 [[ 1.0001000e+00  2.4262429e-03 -5.3907027e-03 -7.7360502e-04
  -7.4419454e-03 -1.0678579e-02]
 [ 2.4262429e-03  1.0001000e+00 -5.4155029e-03 -2.0672032e-03
   4.3277428e-03  3.7399144e-03]
 [-5.3907027e-03 -5.4155029e-03  1.0001000e+00  6.2504560e-03
  -3.7937090e-03  7.3285382e-03]
 [-7.7360502e-04 -2.0672032e-03  6.2504560e-03  1.0001000e+00
  -1.0416691e-02  1.0044558e-02]
 [-7.4419454e-03  4.3277428e-03 -3.7937090e-03 -1.0416691e-02
   1.0001000e+00 -1.3045982e-02]
 [-1.0678579e-02  3.7399144e-03  7.3285382e-03  1.0044558e-02
  -1.3045982e-02  1.0001000e+00]]


## Variance inflation factor

Variance Inflation Factor (VIF) was used to detect multicollinearity between variables. In a linear statistical model, the variance inflation factor (VIF) of a coefficient is equal to the quotient of the variance of that coefficient in a multivariate model and the variance of that coefficient in a model with only one variable. In simple terms, it refers to the ratio of the variance when there is multicollinearity among the explanatory variables (features) to the variance when there is no multicollinearity.

### SSVertVIF
SecretFlow provides `SSVertVIF` for calculating variance inflation factor of Vertical DataFrame using secret sharing.

The vif value of the j-th feature is: $$ VIF_j = (X^TX)^{-1}_{jj}(n-1)var(X_j) $$

Notice:

The analytical solution of matrix inversion in secret sharing is very expensive,
so this method uses Newton iteration to find approximate solution.

When there is multicollinearity in the input dataset, the XTX matrix is not full rank,
and the analytical solution for the inverse of the XTX matrix does not exist.

The VIF results of these linear correlational columns calculated by statsmodels are INF,
indicating that the correlation is infinite.
However, this method will get a large VIF value (>>1000) on these columns,
which can also correctly reflect the strong correlation of these columns.

When there are constant columns in the data, the VIF result calculated by statsmodels is NAN,
and the result of this method is also a large VIF value (>> 1000),
means these columns need to be removed before training.

Therefore, although the results of this method cannot be completely consistent with statemodels
that calculations in plain text, but they can still correctly reflect the correlation of the input data columns.

In [16]:
from secretflow.stats import SSVertVIF 

v_vif = SSVertVIF(spu)
vif = v_vif.vif(vdf)
print(f"vif: \n{vif}")

vif: 
[0.9999169 0.9997679 0.9999169 0.9999169 1.0000659 1.0002149]


## Hypothesis Testing for Regression Coefficients

Linear / logistic regression variable significance test for all features (explanatory variables) use to judge whether the parameter is significant, that is, whether the independent variable can effectively predict the variation of the dependent variable, so as to determine whether the corresponding explanatory variable should be included in the model.

### Hypothesis Testing for linear Regression Coefficients
For linear regression $y=Xω$ (X contains a constant term), use the t-test to test whether the regression term coefficients have zero values.

Assume:

$$ \hat{ω}=(X^T X)^{-1} X^T y=ω+(X^T X)^{-1} X^T ε $$
$$ E(\hat{ω})=ω $$
$$ Var(\hat{ω} )=σ^2 (X^T X)^{-1} $$

In the case where the five assumptions made by OLS are all established, the statistic:
$$ t_j=\frac{\hat{ω}_j- ω_j}{s.e.(ω_j )}=\frac{\hat{ω}_j - ω_j}{\sqrt{\hat{σ}^2 (X^T X)_{jj}^{-1}}}  \sim t_{n-k} $$
where n is sample size, k is feature size.

The null and alternative hypotheses tested are:
$$ \begin{aligned}
& H_0:ω_j=0 (j=1,2,⋯,k) \\ & H_1:ω_j≠0
\end{aligned} $$

Bring the null hypothesis of the test into $t_j$ :

$$ t_j=\frac{\hat{ω}_j}{s.e.(ω_j )}=\frac{\hat{ω}_j}{\sqrt{\hat{σ}^2 (X^T X)_{jj}^{-1}}}  \sim t_{n-k} $$

### Hypothesis Testing for Logistic Regression Coefficients
For logistic regression
$$ P(y=1|x)=π(x)=1/(1+e^{-ωx} ) \\
P(y=0|x)=1-π(x)=1/(1+e^{ωx} ) $$

The null and alternative hypotheses tested are:
$$ \begin{aligned}
& H_0:ω_j=0 (j=1,2,⋯,k) \\
& H_1:ω_j≠0
\end{aligned} $$

Wald test $Wald=(\hat{ω}_k/SE(\hat{ω}_k ) )^2$ fits a chi-square distribution with 1 degree of freedom.

Where $SE(\hat{ω}_k )$ is the standard error of $ω_k$, which is the square root of the diagonal elements of the variance-covariance matrix:
$$ SE(\hat{ω}_k )={Cov(\hat{ω}_{kk})}^{1/2} $$

The variance and covariance matrices of the model parameters are the values ​​of the inverse of the Hessian matrix of the log-likelihood function at $\hat{ω}$:
$$ Cov(\hat{ω}) =H^{-1}=\frac{∂^2 l(ω)}{∂ω^2}|_{\hat{ω}} $$

Where:
$$ H_{kr}=\frac{∂^2l(ω)}{∂ω_k ∂ω_r}=∑_{i=1}^mx_{ik}π(x_i)[π(x_i )-1]x_{ir} $$

The Hessian matrix is ​​expressed as $H=X^T A X$, A is a n*n matrix, where:
$$ \begin{aligned}
& A_{ii} = π(x_{i})[π(x_{i}) - 1] \\
& A_{ij} = 0 , i≠j
\end{aligned} $$

Available:
$$ \begin{aligned}
Wald & = (\hat{ω}_k/SE(\hat{ω}_k ) )^2 \\    
& = \hat{ω}_k^2 /Cov(\hat{ω}_{kk}) \\
& = \hat{ω}_k^2 / H^{-1}_{kk} \\
& = \hat{ω}_k^2 / (X^T \Lambda X )^{-1}_{kk}
\end{aligned} $$

### SSPValue

SecretFlow provides `SSPValue` for calculating P-Value of hypothesis testing using secret sharing.

For linear regression:

  * calculate prediction residuals $\hat{ε}=Xω - y$
  * calculate $\hat{σ}^2=\hat{ε}^T\hat{ε} /(n-k)$
  * get $(X^T X)^{-1}$ by Newton iteration
  * $t^2= ω^2 / \hat{σ}^2(X^T X)_{jj}^{-1}$ 
  * $p =2 * (1 - t_{n-k}.cdf(|t|))$

For logistic regression:

  * calculate $H=X^TAX$
  * get $H^{-1}$ by Newton iteration
  * calculate $z^2=ω^2/H^{-1}_{kk}$
  * $p = 2 * (1 - norm.cdf(|z|))$

In [19]:
from secretflow.stats import SSPValue
from secretflow.ml.linear.ss_sgd import SSRegression

# first, get a LR model
model = SSRegression(spu)
model.fit(
    vdf,
    label_data,
    3,      # epochs
    0.3,    # Learning rate
    64,     # batch_size
    't1',   # sig_type
    'logistic',  # reg_type
    'l2',   # penalty
    0.5,    # l2_norm
)
spu_model = model.save_model()

sspv = SSPValue(spu)
pvalues = sspv.pvalues(vdf, label_data, spu_model)
print(f"logistic pvalue: \n{pvalues}")

logistic pvalue: 
[9.46532264e-08 0.00000000e+00 0.00000000e+00 0.00000000e+00
 8.21281902e-07 0.00000000e+00 0.00000000e+00]
